In [62]:
# Eval somevideo ##  Eval XiJin's First model

import cv2
import numpy as np
from PIL import Image
import sys
sys.path.insert(0,'./caffe/python')
import caffe
import os

import scipy.io as sio    
import matplotlib.pyplot as plt  
plt.rcParams['figure.figsize']=[15,15]
import time
import scipy.misc

################### about network 1 #########################
#INPUT_CHANNEL = 3

Mean = np.array((114.578, 115.294, 108.353), dtype=np.float32)
Std = 0.015625

# load net
DEPLOY = '/home/dalong/Workspace/human_matting/fcn_seg/xinjin_model/VGG16_1c_dep.prototxt'
CAFFEMODEL = '/home/dalong/Workspace/human_matting/fcn_seg/xinjin_model/vgg_co_1labels_minAug_3c_iter_11000.caffemodel'
net = caffe.Net(DEPLOY, CAFFEMODEL, caffe.TEST)
caffe.set_mode_gpu();
caffe.set_device(0);

def predict(img):
    imgshape = img.shape
    img = cv2.resize(img,(192,320))
    img = np.float32(img)
    img -= Mean
    img *= Std
    img = img.transpose((2,0,1))
        
    net.blobs['data'].reshape(1,*img.shape)
    net.blobs['data'].data[...] = img

    # run net and take argmax for prediction
    net.forward()
    out = net.blobs['mask/prob'].data[0]
    out = out.transpose().reshape((32,18,2))[:,:,1]
    out = cv2.resize(out*255,(imgshape[1],imgshape[0]))##368 640
    out[np.where(out<255*0.3)] = 0
    #out[np.where(out>=255*0.3)] = 1
    out = np.array(out, dtype=np.int64)
    return out

############################################################

################### about network 2 #########################
#INPUT_CHANNEL = 4
MEAN = [114.578, 115.294, 108.353]

# load net
DEPLOY2 = './experiment/exper_douyu2700_googlenet_batchsize_4input/deploy_ori.prototxt'
CAFFEMODEL2 = './experiment/exper_douyu2700_googlenet_batchsize_4input/snapshot/douyu_2700_train_iter_40000.caffemodel'
net2 = caffe.Net(DEPLOY2, CAFFEMODEL2, caffe.TEST)

def predict2(img, premask):
    im = img
    imgshape = img.shape
    in_ = np.array(im, dtype=np.float32)
    in_ = in_[:,:,::-1]
    in_ -= np.array(MEAN)
    in_ = in_.transpose((2,0,1))

    imshape = (imgshape[1],imgshape[0], 4)
    in_batch = np.ndarray(shape=(1, imshape[2], imshape[1], imshape[0]), dtype=np.float32)
    in_batch[0,0:3,:,:] = in_.copy()

    # load 4-th channel
    im_pred = premask
    in_ = np.array(im_pred, dtype=np.float32)
    in_ -= 128
    in_batch[0,3,:,:] = in_.copy()
        
    # shape for input (data blob is N x C x H x W), set data
    net2.blobs['data'].reshape(*in_batch.shape)
    net2.blobs['data'].data[...] = in_batch

    # run net and take argmax for prediction
    net2.forward()
    out = net2.blobs['score'].data[0].argmax(axis=0)
    return out

############################################################

video_list = os.listdir('../somevideo/')
for video in video_list:
    if not video[-3:]=='flv':
        continue
    print video

    video_dir = '../somevideo/'+video;
    videoCapture = cv2.VideoCapture(video_dir)

    fps = videoCapture.get(cv2.cv.CV_CAP_PROP_FPS)
    size = (int(videoCapture.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH)), 
            int(videoCapture.get(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT)))

    success, frame = videoCapture.read() 
    frame_show = np.zeros([frame.shape[0], frame.shape[1]*3+20*2,3])

    print size
    print frame_show.shape[1],frame_show.shape[0]

    videoWriter = cv2.VideoWriter(video_dir[:-4]+'_result.avi', cv2.cv.FOURCC(*'XVID') , fps, (frame_show.shape[1],frame_show.shape[0]))
    #videoWriter = cv2.VideoWriter(video_dir[:-4]+'_result.avi', cv2.cv.CV_FOURCC('M', 'J', 'P', 'G') , fps, (frame_show.shape[1],frame_show.shape[0]))

    count = 0
    while success :
        if count%30 == 0:
            print count
        frame_output = frame 
        frame_blur = cv2.blur(frame, (15,15)) 

        frame_mask = predict(frame)
        frame_mask2 = predict2(frame, frame_mask) ## 0/1
        frame_mask2 = np.array(frame_mask2, dtype=np.uint8)
        frame_mask2_bgr = cv2.cvtColor(frame_mask2, cv2.COLOR_GRAY2BGR);

        frame_output = frame*frame_mask2_bgr + frame_blur*(1-frame_mask2_bgr)

        frame_show = np.zeros([frame.shape[0], frame.shape[1]*3+20*2,3], dtype=np.uint8)
        frame_show[0:frame.shape[0],0:frame.shape[1],:] = frame
        frame_show[0:frame.shape[0],frame.shape[1]+20:frame.shape[1]*2+20,:] = frame_output
        frame_show[0:frame.shape[0],frame.shape[1]*2+40:frame.shape[1]*3+40,:] = (frame_mask2_bgr*255)

        videoWriter.write(frame_show) 
        success, frame = videoCapture.read() 
        count +=1

    print count





1488867749.flv
(368, 640)
1144 640
0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
650
1489282820.flv
(368, 640)
1144 640
0
30
60
90
120
150
180
210
240
270
300
330
360
386
1488511789.flv
(368, 640)
1144 640
0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
1488513837.flv
(368, 640)
1144 640
0
30
60
90
120
150
180
210
240
270
300
330
360
390
400
1489249874.flv
(368, 640)
1144 640
0
30
60
90
120
150
180
210
240
270
300
330
360
1489001289.flv
(368, 640)
1144 640
0
30
60
90
120
150
180
200
1488641583.flv
(368, 640)
1144 640
0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
567
1489255039.flv
(368, 640)
1144 640
0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
